# Notebook to Write Qualtrics Surveys

In [3]:
import pandas
import numpy
import simple_search_func as ss
import re
import csv
import json

In [ ]:
# Load up the qsf!
with open('Star_Trek_Fillable.qsf', 'r') as file:
    data = json.load(file)

In [ ]:
choices_dict = {
    'love': ['hello', 'hi', 'are you', 'darling', 'sweetheart', 'honey', 'romance', 'affection', 'passion', 'devotion', 
             'adore', 'cherish', 'fondness', 'embrace', 'beloved', 'cuddle', 'valentine', 'kiss', 'heart', 'soulmate'],
    'sadness': ['meow', 'meow meow', 'meowwww', 'tears', 'gloom', 'despair', 'melancholy', 'sorrow', 'blue', 'grief', 
                'heartache', 'mourn', 'loneliness', 'misery', 'pain', 'regret', 'weep', 'hurt', 'lost', 'downcast'],
    'kiss': ['smoocher', 'kombucher', 'peck', 'smooch', 'liplock', 'mwah', 'pucker', 'osculation', 'romantic', 'passionate',
             'sweet', 'gentle', 'soft', 'warm', 'butterfly', 'cheek', 'forehead', 'eskimo', 'lovebite', 'chaste'],
    'happiness': ['joy', 'smile', 'laughter', 'cheer', 'delight', 'bliss', 'glee', 'ecstatic', 'merry', 'euphoria', 
                  'sunny', 'content', 'radiant', 'jubilant', 'bubbly', 'grateful', 'optimistic', 'carefree', 'vivacious', 'buoyant'],
    'anger': ['rage', 'fury', 'wrath', 'outrage', 'resentment', 'bitterness', 'spite', 'grudge', 'irritation', 'annoyance',
              'exasperation', 'hostility', 'hatred', 'indignation', 'fuming', 'seething', 'mad', 'infuriated', 'irate', 'tempest'],
    'fear': ['terror', 'panic', 'dread', 'fright', 'horror', 'alarm', 'anxiety', 'nervous', 'uneasy', 'apprehension',
             'timid', 'startle', 'tremble', 'phobia', 'shiver', 'quiver', 'spooked', 'petrified', 'chill', 'paranoia'],
    'surprise': ['shock', 'amazement', 'astonishment', 'wonder', 'startled', 'stunned', 'flabbergasted', 'aghast', 'dumbfounded', 'jolted',
                 'marvel', 'unexpected', 'perplexed', 'bewildered', 'gobsmacked', 'astounded', 'overwhelmed', 'speechless', 'baffled', 'staggered'],
    'friendship': ['bond', 'companion', 'ally', 'trust', 'loyal', 'camaraderie', 'mate', 'bro', 'sisterhood', 'brotherhood',
                   'affection', 'partnership', 'support', 'understanding', 'team', 'respect', 'connect', 'harmony', 'together', 'kindred'],
    'jealousy': ['envy', 'resent', 'covet', 'green-eyed', 'possessive', 'suspicion', 'mistrust', 'distrust', 'bitterness', 'spite',
                 'comparison', 'insecurity', 'longing', 'competitiveness', 'inferiority', 'grudge', 'fuming', 'anger', 'watchful', 'wary'],
    'nostalgia': ['memories', 'past', 'bittersweet', 'longing', 'reminisce', 'yearning', 'flashback', 'old days', 'sentimental', 'melancholy',
                  'childhood', 'throwback', 'wistful', 'retro', 'antique', 'classic', 'golden days', 'vintage', 'heartwarming', 'timeless']
}

topics = list(choices_dict.keys())# will be dict keys
c_topics = topics.copy()
surveyid = 'newstring'

# iterate
for x in data['SurveyElements']:
    if type(x['Payload']) != dict:
        continue
    if 'QuestionType' not in x['Payload'].keys():
        continue
    x['SurveyID']= surveyid
    if x['Payload']['QuestionType']=='PGR': # group rank, influences others
        text = topics.pop()
        old_text = x['SecondaryAttribute']
        new_q = re.sub(r'"(.*?)"', f'"{old_text}"', text)
        x['Payload']['QuestionDescription']= new_q
        x['SecondaryAttribute'] = new_q
        x['Payload']['QuestionText']= new_q
        choices = choices_dict[text]
        new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
        x['Payload']['Choices']=new_choices
    if x['Payload']['QuestionType']=='RO': # rank only, depends on other
        # update Secondary Attribute: replace whatever is in ""
        text = c_topics.pop()
        old_text = x['SecondaryAttribute']
        new_q = re.sub(r'"(.*?)"', f'"{old_text}"', text)
        x['SecondaryAttribute'] = new_q
        # update ['Payload']['QuestionText'] replace whatever is in ""
        x['Payload']['QuestionText']= new_q
        x['Payload']['QuestionDescription']= new_q

# save
with open('test.qsf', 'w') as json_file:
    json.dump(data, json_file) 

In [ ]:
# make this a function

def new_survey_file(choices_dict:dict,file_name:str,template_name:str='Star_Trek_Fillable.qsf'):
    with open(template_name, 'r') as file:
        data = json.load(file)
    topics = list(choices_dict.keys())# will be dict keys
    c_topics = topics.copy()
    surveyid = 'newstring'

    # iterate
    for x in data['SurveyElements']:
        if type(x['Payload']) != dict:
            continue
        if 'QuestionType' not in x['Payload'].keys():
            continue
        x['SurveyID']= surveyid
        if x['Payload']['QuestionType']=='PGR': # group rank, influences others
            text = topics.pop()
            old_text = x['SecondaryAttribute']
            new_q = re.sub(r'"(.*?)"', f'"{old_text}"', text)
            x['Payload']['QuestionDescription']= new_q
            x['SecondaryAttribute'] = new_q
            x['Payload']['QuestionText']= new_q
            choices = choices_dict[text]
            new_choices = {str(x+1):{'Display': choices[x]} for x in range(len(choices))}
            x['Payload']['Choices']=new_choices
        if x['Payload']['QuestionType']=='RO': # rank only, depends on other
            # update Secondary Attribute: replace whatever is in ""
            text = c_topics.pop()
            old_text = x['SecondaryAttribute']
            new_q = re.sub(r'"(.*?)"', f'"{old_text}"', text)
            x['SecondaryAttribute'] = new_q
            # update ['Payload']['QuestionText'] replace whatever is in ""
            x['Payload']['QuestionText']= new_q
            x['Payload']['QuestionDescription']= new_q

    # save
    with open(file_name, 'w') as json_file:
        json.dump(data, json_file) 